In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.metrics import f1_score
from os import path
from os import listdir
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
from math import log2, e
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
f = open("drive/MyDrive/author_quotes.txt", "r")
n = 36165
data = [0] * n
s = set()
for i in range(n):
  words = f.readline().strip()
  words = words.lower()
  data[i] = words[:-1]
  for x in words:
    s.add(x)
print(s)

{'v', '#', ':', '9', '(', '!', '/', 'r', '+', '&', 'x', '5', 'o', '7', "'", '6', 'q', 'w', 'a', '-', ';', '—', 'g', ',', 'f', 'e', 'h', 't', 'b', '%', 's', '*', '1', '£', ' ', 'n', '=', 'd', '8', 'u', ')', 'z', 'c', '0', '2', '.', '?', 'i', 'k', '3', 'y', '"', '$', '4', 'm', 'p', 'j', 'l', '’'}


In [ ]:
todel = ",£;’)=\"?*(/\'$!-&+#—.:%"
for c in todel:
  for i in range(len(data)):
    data[i] = data[i].replace(c, "")
s = set()
for x in data:
  for symb in x:
    s.add(symb)
s = (list(s))
s.sort()
print(s)

[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
index = 0
symb_to_int = dict()
int_to_symb = dict()
for c in s:
  if (c not in symb_to_int):
    symb_to_int[c] = index
    int_to_symb[index] = c
    index += 1

In [ ]:
def to_num(seq):
  ans = []
  for j in seq:
    ans.append(symb_to_int[j])
  return ans

real_data = [0] * len(data)
for i in range(len(data)):
  real_data[i] = to_num(data[i])
print(real_data[0])
print(data[0])

[19, 16, 0, 35, 25, 31, 0, 22, 19, 32, 15, 0, 30, 25, 0, 12, 15, 0, 11, 0, 18, 31, 24, 14, 28, 15, 14, 0, 19, 0, 33, 11, 24, 30, 0, 30, 25, 0, 22, 19, 32, 15, 0, 30, 25, 0, 12, 15, 0, 11, 0, 18, 31, 24, 14, 28, 15, 14, 0, 23, 19, 24, 31, 29, 0, 25, 24, 15, 0, 14, 11, 35, 0, 29, 25, 0, 19, 0, 24, 15, 32, 15, 28, 0, 18, 11, 32, 15, 0, 30, 25, 0, 22, 19, 32, 15, 0, 33, 19, 30, 18, 25, 31, 30, 0, 35, 25, 31]
if you live to be a hundred i want to live to be a hundred minus one day so i never have to live without you


In [ ]:
seq = 20
X_Y = []
for i in range(len(data)):
  for j in range(len(data[i])):
    if (j + seq + 1 < len(data[i])):
      X_Y.append((real_data[i][j:j + seq], real_data[i][j + 1:j + seq + 1]))
print(X_Y[0])

([19, 16, 0, 35, 25, 31, 0, 22, 19, 32, 15, 0, 30, 25, 0, 12, 15, 0, 11, 0], [16, 0, 35, 25, 31, 0, 22, 19, 32, 15, 0, 30, 25, 0, 12, 15, 0, 11, 0, 18])


In [ ]:
class LSTM(torch.nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()
    self.features = len(s)
    self.embedd = self.features
    self.layers = 4
    self.embedding = torch.nn.Embedding(
        num_embeddings=self.features,
        embedding_dim=self.embedd,
    )
    self.lstm = torch.nn.LSTM(
        input_size=self.embedd,
        hidden_size=self.embedd,
        num_layers=self.layers
    )
    self.fc = torch.nn.Linear(self.embedd, len(s))
  def forward(self, x, prev_state):
    embed = self.embedding(x)
    output, state = self.lstm(embed, prev_state)
    logits = self.fc(output)
    return logits, state

  def init_state(self):
    return (torch.zeros(self.layers, seq, self.embedd), 
            torch.zeros(self.layers, seq, self.embedd))

class Dataset(torch.utils.data.Dataset):
  def __init__(self, data):
    self.data = data
      
  def __len__(self):
    return len(self.data)

  def __getitem__(self, i):
    return (torch.LongTensor(self.data[i][0]), torch.LongTensor(self.data[i][1]))

NameError: ignored

In [ ]:
model = LSTM().to(device)
batch_size = 1000

In [ ]:
def train(model):
  dataset = Dataset(X_Y)
  dataloader = DataLoader(dataset, batch_size, num_workers=6)
  model.train()
  for _, (x, y) in tqdm(enumerate(dataloader)):
    h, c = model.init_state()
    optimizer.zero_grad()
    x = x.to(device)
    y = y.to(device)
    h = h.to(device)
    c = c.to(device)
    y_pred, (h, c) = model(x, (h, c))
    
    loss = criterion(y_pred.transpose(1, 2), y)
    loss.backward()
    optimizer.step()
    print(loss.item())

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train(model)

2it [00:00,  5.61it/s]

2.841902256011963
2.811788320541382


4it [00:00,  6.55it/s]

2.879502058029175
2.849597692489624


6it [00:00,  7.16it/s]

...

3814it [08:15,  8.01it/s]

1.7976861000061035
1.8587136268615723


3816it [08:15,  8.11it/s]

1.898675560951233
1.8990365266799927
2.0917441844940186


3817it [08:16,  7.69it/s]


In [ ]:
def predict(pref, model, amount):
  model.eval()
  ans = pref
  tens = []
  for j in pref:
    tens.append(symb_to_int[j])
  h, c = model.init_state()
  while (len(ans) < amount): 
    x = torch.tensor([tens[-seq:]])
    x = x.to(device)
    h = h.to(device)
    c = c.to(device)
    y_pred, (h, c) = model(x, (h, c))
    p = torch.nn.functional.softmax(y_pred[0][-1], dim=0).cpu().detach().numpy()
    index = np.random.choice(len(y_pred[0][-1]), p=p)
    tens.append(index)
    ans += int_to_symb[index]

  return ans

In [ ]:
for i in range(4):
  print(predict('during this process ', model, 777))

during this process that me fut in me i do word as wound so mud lols ofn of the kdelants elcerny creocctaved a lofses and exemy lowisg ow be to we guword yrios wraf wore the a sgenate it um and seat whet and of doling a fousate non gay seed as put buse hoersoys trrad aclaciss of cate iqtents of coernint the susu0 the lel eve thatanroting fraleds the wiven a mey if cinl stive woalt onon yoir mitinges statil and imllelt suvins the natioe be go to and we paralibrhing wart sflol trice cifgans thripe bed plincaiges oe folt exd that 1m was and shance fom wended and going it morsing goigth ony e5 no chigk thit and for i wriinlurinpd insed the that the datan the a finsl taned ever and are fortectfuces youed theyeling in a ficalve thardnes bosady wol tole letfy reserty arlets
during this process art hhass neges i jenbol welt tomelhiticirans tha hhoe intal that mens mach al i vhu the do waote traty is dasite a tecles to bid of with thiw neple a sorl so milsy that neveers beperitinc the bust sume

In [ ]:
class MARK:
  def __init__(self, sz):
    self.autom = dict()
    self.autom_sz = dict()
    self.size = sz

  def train(self, text):
    for i in range(len(text)):
      if (i + self.size >= len(text)):
        break
      key = text[i:i + self.size]
      symb = text[i + self.size]
      if (key in self.autom):
        self.autom[key][symb_to_int[symb]] += 1
        self.autom_sz[key] += 1
      else:
        self.autom[key] = np.zeros(len(s))
        self.autom[key][symb_to_int[symb]] += 1
        self.autom_sz[key] = 1

  def predict(self, pref, size):
    ans = pref
    prob = dict()
    for key in self.autom:
      prob[key] = np.zeros(len(s))
      for j in range(len(s)):
        if (self.autom_sz[key] > 0):
          prob[key][j] = self.autom[key][j] / self.autom_sz[key]
      #print(prob[key])
    while (len(ans) < size):
      key = ans[-self.size:]
      next = np.random.choice(len(s), p=prob[key])
      ans += int_to_symb[next]
    return ans

mark = MARK(4)

In [ ]:
for dat in data:
  mark.train(dat)

In [ ]:
for i in range(3):
  print(mark.predict("during this process", 100))

during this process what get actors old reservitude i waking refore good auschwarzenegrind at to wor
during this process so love summer been on of courselves you can housewives work on to do your job w
during this process it was woman collected with myself rest case i ever to say night of our in a lot
